
# AI Agent: Textbook to Notebook Notes Generator
This program converts a textbook PDF into concise, structured academic notes.
It uses the local Llama3:8b model (via Ollama) to generate notes, and Gemini API to evaluate them.
Finally, it creates a downloadable PDF using the FPDF library.


In [72]:
import os
import json
import gradio as gr
import textwrap
import asyncio
import markdown_it
from dotenv import load_dotenv
from openai import AsyncOpenAI
from pypdf import PdfReader
from pydantic import BaseModel
from fpdf import FPDF

In [73]:
# Load environment variables from a .env file
load_dotenv(override=True)

True

In [74]:
# Download and start the Llama3 model server locally
!ollama pull llama3:8b
!ollama serve

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling 6a0746a1ec1a: 100% ▕██████████████████▏ 4.7 GB                         
pulling 4fa551d4f938: 100% ▕██████████████████▏  12 KB                         
pulling 8ab4849b038c: 100% ▕██████████████████▏  254 B                         
pulling 577073ffcc6c: 100% ▕██████████████████▏  110 B                         
pulling 3f8eb4da87fa: 100% ▕██████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 
Error: listen tcp 127.0.0.1:11434: bind: Only one usage of each socket address (protocol/network address/port) is normally permitted.


In [75]:
# Fetch API keys for OpenAI (optional) and Google Gemini
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

# Display partial API keys for confirmation
if openai_api_key:
    print(f"OpenAI API Key found, starting with: {openai_api_key[:8]}...")
else:
    print("OpenAI API Key not found. Please set it in your .env file.")

if google_api_key:
    print(f"Google API Key found, starting with: {google_api_key[:8]}...")
else:
    print("Google API Key not found. Please set it in your .env file.")

OpenAI API Key found, starting with: sk-proj-...
Google API Key found, starting with: AIzaSyDn...


In [76]:
# Create asynchronous clients for both Ollama (Llama3) and Gemini models
ollama_client = AsyncOpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

openai_client = AsyncOpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1"
)

gemini_client = AsyncOpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [77]:
# Define a data model for Gemini’s evaluation output
class Evaluation(BaseModel):
    """Defines evaluator output."""
    is_acceptable: bool
    feedback: str

In [78]:
async def generate_notes(text_chunk: str, retries=2, feedback=""):
    """
    Generate structured notes for a given text chunk using Llama3:8b.
    If Gemini evaluation fails, it retries with feedback for self-correction.
    """

    # System prompt instructs the model to act as an academic assistant
    system_prompt = (
        "You are an expert academic assistant. "
        "Read the provided text and produce well-organized, clear Markdown notes. "
        "Focus on key concepts, definitions, and main ideas. "
        "Keep the language simple but precise."
    )

    # If previous output was rejected, include feedback for improvement
    if feedback:
        user_prompt = (
            f"The previous notes were not acceptable. Improve them using this feedback:\n"
            f"{feedback}\n\nOriginal Text:\n{text_chunk}"
        )
    else:
        user_prompt = f"Generate concise academic notes for the following text:\n{text_chunk}"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    # Try generating notes using the Ollama (Llama3) model
    try:
        response = await ollama_client.chat.completions.create(
            model="llama3:8b",
            messages=messages,
        )
        notes = response.choices[0].message.content
    except Exception as e:
        print(f"Error during Ollama generation: {e}")
        return f"Error generating notes: {e}"

    # Evaluate generated notes using Gemini
    if retries > 0:
        evaluation = await evaluate_notes(text_chunk, notes)
        if not evaluation.is_acceptable:
            print(f"Evaluation failed. Retrying with feedback: {evaluation.feedback}")
            return await generate_notes(text_chunk, retries - 1, evaluation.feedback)
        else:
            print("Evaluation passed.")

    return notes

In [79]:
async def evaluate_notes(text_chunk: str, notes: str) -> Evaluation:
    """
    Evaluates generated notes using ChatGPT-compatible free model from OpenRouter.
    """
    prompt = (
        "You are an evaluator. Rate the following notes based on accuracy, "
        "clarity, and completeness. Return JSON only with keys:\n"
        "is_acceptable (boolean), feedback (string).\n\n"
        f"--- Original Text ---\n{text_chunk}\n\n"
        f"--- Notes ---\n{notes}"
    )

    try:
        response = await openai_client.chat.completions.create(
            model="openai/gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
        )
        content = response.choices[0].message.content.strip()
        data = json.loads(content)
        return Evaluation(**data)

    except Exception as e:
        print(f"Evaluation error: {e}")
        return Evaluation(is_acceptable=True, feedback=f"Evaluation failed: {e}")

In [80]:
def chunk_text(text: str, max_chars: int = 2500):
    """
    Splits long text into smaller chunks for easier processing.
    Each chunk is roughly 'max_chars' characters long.
    """
    return textwrap.wrap(text, width=max_chars, break_long_words=False, replace_whitespace=False)

In [81]:
def create_pdf_file(notes_markdown: str, source_filename: str) -> str:
    """
    Converts generated Markdown notes into a formatted PDF using FPDF2.
    Falls back to a Markdown file if PDF generation fails.
    """
    title = os.path.splitext(os.path.basename(source_filename))[0].replace('_', ' ').title()
    output_filename = f"{os.path.splitext(source_filename)[0]}_notes.pdf"
    
    pdf = FPDF()
    pdf.add_page()

    # Add title
    pdf.set_font("Arial", "B", 18)
    pdf.cell(0, 10, f"Notes for {title}", 0, 1, "C")
    pdf.ln(10)

    # Add body text (Markdown supported in FPDF2)
    pdf.set_font("Arial", "", 11)
    
    try:
        pdf.write_markdown(notes_markdown)
        pdf.output(output_filename)
        print(f"PDF created successfully: {output_filename}")
        return output_filename
    except Exception as e:
        print(f"Error creating PDF with fpdf2: {e}")
        md_filename = f"{os.path.splitext(source_filename)[0]}_notes.md"
        with open(md_filename, "w", encoding="utf-8") as f:
            f.write(f"# Notes for {title}\n\n{notes_markdown}")
        print(f"As a fallback, Markdown file saved: {md_filename}")
        return md_filename

In [82]:
async def process_textbook(file, progress=gr.Progress()):
    """
    Main orchestrator:
    1. Extracts all text from the uploaded PDF.
    2. Splits it into chunks.
    3. Generates notes for each chunk using Llama3.
    4. Evaluates quality using Gemini.
    5. Combines all notes and saves as a PDF.
    """
    if file is None:
        return None

    pdf_file_path = file.name
    reader = PdfReader(pdf_file_path)
    num_pages = len(reader.pages)
    
    print(f"Extracting text from {num_pages} pages...")
    progress(0, desc="Step 1/3: Extracting Text...")
    full_text = ""

    # Extract text page by page
    for i, page in enumerate(reader.pages):
        progress((i + 1) / num_pages, desc=f"Extracting from Page {i + 1}/{num_pages}")
        page_text = page.extract_text()
        if page_text:
            full_text += page_text + "\n"

    if not full_text.strip():
        print("No text could be extracted from the PDF.")
        return None

    print(f"Text extraction complete. Total characters: {len(full_text)}")
    
    # Split text and process in chunks
    chunks = chunk_text(full_text)
    num_chunks = len(chunks)
    all_notes = []
    
    print(f"Generating notes from {num_chunks} text chunks...")
    progress(0, desc="Step 2/3: Generating Notes...")
    for i, chunk in enumerate(chunks):
        progress((i + 1) / num_chunks, desc=f"Processing Chunk {i + 1}/{num_chunks}")
        notes_chunk = await generate_notes(chunk)
        all_notes.append(notes_chunk)
        
    combined_notes = "\n\n---\n\n".join(all_notes)

    progress(1, desc="Step 3/3: Creating PDF...")
    pdf_path = create_pdf_file(combined_notes, pdf_file_path)
    return pdf_path


In [83]:
async def create_notes_interface(file, progress=gr.Progress(track_tqdm=True)):
    """
    Wrapper for Gradio interface to handle file upload and call the main process.
    """
    if file is not None:
        return await process_textbook(file, progress)
    return "Please upload a textbook to begin."

# Define the Gradio interface
iface = gr.Interface(
    fn=create_notes_interface,
    inputs=gr.File(label="Upload Textbook (PDF)"),
    outputs=gr.File(label="Download Generated Notes (.pdf)"),
    title="AI Textbook → PDF Notes Generator",
    description=(
        "Upload a textbook in PDF format. The local llama3:8b model generates a cohesive summary "
        "of the entire book, Gemini API evaluates its quality, and you receive a downloadable PDF."
    ),
)

# Run the Gradio app
if __name__ == "__main__":
    iface.launch(server_name="127.0.0.1", share=False)

* Running on local URL:  http://127.0.0.1:7888
* To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Upendra\Git Hub\Git Hub -- K-Upendra-7\abcd-agentic-training-vnr-upendra\AI-Agent-Textbook-Notebook\.venv\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        self.scope, self.receive, self.send
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Upendra\Git Hub\Git Hub -- K-Upendra-7\abcd-agentic-training-vnr-upendra\AI-Agent-Textbook-Notebook\.venv\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Upendra\Git Hub\Git Hub -- K-Upendra-7\abcd-agentic-training-vnr-upendra\AI-Agent-Textbook-Notebook\.venv\Lib\site-packages\fastapi\applications.py", line 1133, in __call__
    await super().__call__(scope, rec

Extracting text from 11 pages...
Text extraction complete. Total characters: 19898
Generating notes from 8 text chunks...
Evaluation error: Error code: 401 - {'error': {'message': 'No cookie auth credentials found', 'code': 401}}
Evaluation passed.
Evaluation error: Error code: 401 - {'error': {'message': 'No cookie auth credentials found', 'code': 401}}
Evaluation passed.
Evaluation error: Error code: 401 - {'error': {'message': 'No cookie auth credentials found', 'code': 401}}
Evaluation passed.
Evaluation error: Error code: 401 - {'error': {'message': 'No cookie auth credentials found', 'code': 401}}
Evaluation passed.
Evaluation error: Error code: 401 - {'error': {'message': 'No cookie auth credentials found', 'code': 401}}
Evaluation passed.
Evaluation error: Error code: 401 - {'error': {'message': 'No cookie auth credentials found', 'code': 401}}
Evaluation passed.
Evaluation error: Error code: 401 - {'error': {'message': 'No cookie auth credentials found', 'code': 401}}
Evaluati

C:\Users\kolla\AppData\Local\Temp\ipykernel_20056\865379566.py:13: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", "B", 18)
C:\Users\kolla\AppData\Local\Temp\ipykernel_20056\865379566.py:14: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f"Notes for {title}", 0, 1, "C")
C:\Users\kolla\AppData\Local\Temp\ipykernel_20056\865379566.py:18: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", "", 11)
